In [1]:
# Import libraries
import json
import pandas as pd
from pandas import DataFrame
from pandas.io.json import json_normalize #package for flattening json in pandas df

In [3]:
file_path = '../test_data/weather_obs-1.json'

#load json object
with open(file_path) as f:
    d = json.load(f)

# Parse JSON file into a DataFrame
raw_data = json_normalize(d['SiteRep'])
raw_data

,DV.Location,DV.dataDate,DV.type,Wx.Param
0,"[{'i': '3002', 'lat': '60.749', 'lon': '-0.854...",2016-06-21T16:00:00Z,Obs,"[{'name': 'G', 'units': 'mph', '$': 'Wind Gust..."


In [4]:
# Parse the DV columns from JSON file
location = json_normalize(data=d['SiteRep']['DV'], 
                          record_path=['Location'],
                          meta=['dataDate', 'type'])

# new = DataFrame(pd.json.loads(location.head(1).to_json()))
a = DataFrame(location.iloc[18])
# location.head(2)

# isinstance(location.iloc[19]['Period'], list)
len(location)

145

In [5]:
a = DataFrame(location.iloc[18])
# location.head(2)

a.T

,Period,continent,country,elevation,i,lat,lon,name,dataDate,type
18,"[{'type': 'Day', 'value': '2016-06-20Z', 'Rep'...",EUROPE,SCOTLAND,933.0,3072,56.879,-3.42,CAIRNWELL,2016-06-21T16:00:00Z,Obs


In [6]:
b = json_normalize(data=a['Period'], record_path=['Rep'],
              meta=['type', 'value'])
b

KeyError: 'Period'

In [7]:
# COPY META COLUMNS TO PERIOD RESULTS DATAFRAME
# a: 1-row DataFrame containing Period and meta columns
# b: results DataFrame
def copy_meta_to_period(a, b):
    headers = list(a)
    for i in range(1,len(headers)):
        b[str(headers[i])] = a.iloc[0][i]
    return b

def handle_lists(x, i):
    a = DataFrame(x.iloc[i].to_dict())
    b = json_normalize(data=a['Period'], record_path=['Rep'], meta=['type', 'value'])
    return a, b

def handle_objects(x, i):
    a = DataFrame(x.iloc[i])
    b = json_normalize(data=a.T['Period'], record_path=['Rep'], meta=['type', 'value'])
    return a, b

result = DataFrame()
# Loop over all rows of location
for i in range (0, len(location)):
     
    if (isinstance(location.iloc[i]['Period'], list)): # Handle lists
        [a, b] = handle_lists(location, i)
    else:                                              # Handle objects
        [a, b] = handle_objects(location, i)
    
    if (len(result) == 0):
        result = copy_meta_to_period(a, b)
    else:
        result = result.append(copy_meta_to_period(a, b), ignore_index=True)
    
result.to_csv()

,$,D,Dp,G,H,P,Pt,S,T,V,...,continent,country,dataDate,elevation,i,lat,lon,name,type,value
0,960,WSW,10.2,32,82.0,1000,R,23,13.2,20000,...,EUROPE,SCOTLAND,2016-06-21T16:00:00Z,15.0,3002,60.749,-0.854,BALTASOUND,Obs,2016-06-20Z
1,1020,W,10.4,31,82.6,1001,R,22,13.3,21000,...,EUROPE,SCOTLAND,2016-06-21T16:00:00Z,15.0,3002,60.749,-0.854,BALTASOUND,Obs,2016-06-20Z
2,1080,W,10.1,30,83.6,1002,R,22,12.8,16000,...,EUROPE,SCOTLAND,2016-06-21T16:00:00Z,15.0,3002,60.749,-0.854,BALTASOUND,Obs,2016-06-20Z
3,1140,WSW,9.8,30,87.5,1002,R,23,11.8,18000,...,EUROPE,SCOTLAND,2016-06-21T16:00:00Z,15.0,3002,60.749,-0.854,BALTASOUND,Obs,2016-06-20Z
4,1200,WSW,9.7,29,88.1,1003,R,18,11.6,17000,...,EUROPE,SCOTLAND,2016-06-21T16:00:00Z,15.0,3002,60.749,-0.854,BALTASOUND,Obs,2016-06-20Z
5,1260,WSW,9.7,NaN,88.7,1004,R,19,11.5,17000,...,EUROPE,SCOTLAND,2016-06-21T16:00:00Z,15.0,3002,60.749,-0.854,BALTASOUND,Obs,2016-06-20Z
6,1320,WSW,9.8,NaN,92.9,1005,R,14,10.9,14000,...,EUROPE,SCOTLAND,2016-06-21T16:00:00Z,15.0,3002,60.749,-0.854,BALTASOUND,Obs,2016-06-20Z
7,1380,WSW,9.8,NaN,91.1,1005,R,18,11.2,16000,...,EUROPE,SCOTLAND,2016-06-21T16:00:00Z,15.0,3002,60.749,-0.854,BALTASOUND,Obs,2016-06-20Z
8,0,WSW,9.4,NaN,90.4,1006,R,17,10.9,16000,...,EUROPE,SCOTLAND,2016-06-21T16:00:00Z,15.0,3002,60.749,-0.854,BALTASOUND,Obs,2016-06-21Z
9,60,WSW,8.4,NaN,90.9,1006,R,15,9.8,17000,...,EUROPE,SCOTLAND,2016-06-21T16:00:00Z,15.0,3002,60.749,-0.854,BALTASOUND,Obs,2016-06-21Z


In [169]:
# period = json_normalize(data=location.head(1)['Period'], record_path=['Rep'])
# period

def normalize_period(df):
    return json_normalize(data=df['Period'].to_dict(), record_path=[0, 'Rep'],
                          meta=[['0', 'type'], ['0', 'value']])

location.head(1).apply(normalize_period)

KeyError: ('Period', 'occurred at index Period')

In [174]:
period = json_normalize(data=d['SiteRep']['DV'], record_path=['Location','Period', 'Rep'],
                       meta=['dataDate', 
                             ['Location', 'continent'],
                             ['Location', 'country'],
                             ['Location', 'name'],
                             ['Location', 'elevation'],
                             ['Location', 'lat'],
                             ['Location', 'lon'],
                             ['Location', 'i'],
                             ['Location', 'Period', 'type'],
                             ['Location', 'Period', 'value'],
                             'type'])

period

TypeError: string indices must be integers